In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)

In [ ]:
df = pd.read_csv('./GSE150150_raw_counts.csv')

In [ ]:
columns = ["Trunk E9.5", "Trunk E10.5", "DRG E9.5", "DRG E10.5", "DRG E11.5", "DRG E12.5", "Cranio E12.5"]
idMappings = {
    "SS2_15_0085": "Trunk E9.5",
    "SS2_18_107": "Trunk E10.5",
    "SS2_18_108": "Trunk E10.5",
    "SS2_18_109": "Trunk E10.5",
    "SS2_16_767": "DRG E9.5",
    "SS2_17_435": "DRG E9.5",
    "SS2_16_688": "DRG E10.5",
    "SS2_16_769": "DRG E10.5",
    "SS2_17_432": "DRG E10.5",
    "SS2_17_433": "DRG E10.5",
    "SS2_16_385": "DRG E11.5",
    "SS2_16_384": "DRG E12.5",
    "SS2_15_0073": "Cranio E12.5",
    "SS2_15_0169": "Cranio E12.5"
}

In [ ]:
print("Gene Count: ", len(df))
sampleRow = df.sample()
print("Sample Row: ", sampleRow)
print("Sample Gene: ", sampleRow.iloc[0,0])

In [ ]:
import numbers
df.columns = [idMappings[col.split(":")[0]] if col.split(":")[0] in idMappings else col for col in df.columns]
dfParsed = df.groupby(df.columns, axis=1).apply(lambda x: x.mean(axis=1) if isinstance(x.iloc[0,0], numbers.Number) else x.iloc[:,0])
dfParsed = dfParsed[dfParsed.columns[5:].tolist() + dfParsed.columns[:5].tolist()]

print("Grouped Gene Count: ", len(dfParsed))
sampleRow = dfParsed.sample()
print("Grouped Sample Row: ", sampleRow)
print("Grouped Sample Gene: ", sampleRow.iloc[0,0])

# Leiden Algorithm

In [2]:
import leidenalg
import igraph as ig

In [3]:
G = ig.Graph.Erdos_Renyi(100, 0.1)

In [5]:
part = leidenalg.find_partition(G, leidenalg.ModularityVertexPartition)
print(part)

Clustering with 100 elements and 6 clusters
[0] 0, 1, 17, 18, 19, 20, 21, 29, 32, 39, 42, 50, 51, 56, 64, 65, 68, 71, 72,
    79, 82, 90, 92, 98
[1] 7, 10, 15, 26, 30, 31, 35, 38, 47, 48, 55, 62, 67, 77, 78, 87, 88, 97, 99
[2] 2, 14, 25, 27, 33, 34, 43, 44, 49, 52, 57, 59, 66, 76, 84, 91
[3] 4, 5, 6, 11, 16, 24, 36, 37, 40, 46, 53, 54, 58, 74, 85
[4] 9, 12, 13, 23, 28, 61, 69, 73, 80, 86, 89, 93, 94
[5] 3, 8, 22, 41, 45, 60, 63, 70, 75, 81, 83, 95, 96
